In [1]:
import warnings
warnings.simplefilter('ignore')

#from functools import partial

import pandas as pd
import numpy as np
import pandas_profiling as pp

#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, train_test_split, KFold
from sklearn.metrics import roc_auc_score, classification_report
#from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy.stats import rankdata
from sklearn.svm import SVC

import lightgbm as lgb
from catboost import Pool, CatBoostClassifier
%matplotlib inline
import matplotlib as plt
import matplotlib.style as stl
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

plt.rcParams['figure.figsize'] = 16, 10



In [2]:
from os import cpu_count

In [3]:
#from xgboost import XGBClassifier

In [4]:
import xgboost as xgb

In [5]:
n_jobs = max(cpu_count()-1, 1)

In [6]:
train = pd.read_csv('train_cleansed.csv')
test = pd.read_csv('test_cleansed.csv')

In [7]:
pd.options.display.max_columns = None

In [8]:
X = train.drop(['0'], axis=1)
y = train['0']

# XGBOOST

## Holdout validation 

In [9]:
#skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [912]:

from sklearn.feature_selection import SelectFromModel
from numpy import sort

In [1249]:
xgb_train = xgb.DMatrix(X, y, feature_names=X.columns)

In [1250]:
parameters = {
     'base_score' : 0.7,
    'eta': 0.1,
    'num_round' : 120,
    'nthread': 4,
    # regularization parameters
    'max_depth' : 4, 
    'max_leaves':32,
    #min_child_weight = 0.05,
    'max_delta_step' :0,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'refresh_leaf' :1,
    'eval_metric':'auc',
    'tree_method' :'hist'
    
    
}

In [1251]:
num_rounds = 10000

In [1252]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [1253]:
results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=10,
                 folds=skf, verbose_eval=1)

[0]	train-auc:0.671133+0.00600566	test-auc:0.660656+0.00693095
[1]	train-auc:0.69169+0.00433044	test-auc:0.677087+0.00399613
[2]	train-auc:0.69946+0.00286043	test-auc:0.685278+0.00439935
[3]	train-auc:0.705996+0.0039537	test-auc:0.688373+0.00500111
[4]	train-auc:0.711048+0.000927741	test-auc:0.693732+0.00706777
[5]	train-auc:0.715668+0.0020433	test-auc:0.69813+0.00574483
[6]	train-auc:0.716829+0.0016739	test-auc:0.698778+0.00603914
[7]	train-auc:0.718602+0.00166974	test-auc:0.700407+0.00515107
[8]	train-auc:0.722291+0.0017573	test-auc:0.70233+0.0043701
[9]	train-auc:0.723538+0.00179023	test-auc:0.702717+0.00502645
[10]	train-auc:0.725776+0.00287805	test-auc:0.704099+0.00474068
[11]	train-auc:0.728279+0.00283888	test-auc:0.705651+0.00379832
[12]	train-auc:0.730231+0.00153283	test-auc:0.707165+0.00384038
[13]	train-auc:0.733445+0.00116828	test-auc:0.708589+0.00435622
[14]	train-auc:0.734996+0.00160866	test-auc:0.710049+0.00468316
[15]	train-auc:0.736652+0.00145499	test-auc:0.710405+0.003

In [1254]:
results.head()

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.660656,0.006931,0.671133,0.006006
1,0.677087,0.003996,0.691690,0.004330
2,0.685278,0.004399,0.699460,0.002860
3,0.688373,0.005001,0.705996,0.003954
4,0.693732,0.007068,0.711048,0.000928


In [1255]:
print(max(results['test-auc-mean']), len(results['test-auc-mean']))


0.731483 107


In [1256]:
model = xgb.XGBClassifier(binary='binary:logistic',
    base_score = 0.7,
    eta= 0.1,
    num_round = 107,
    nthread= 4,
    # regularization parameters
    max_depth = 4, 
    max_leaves= 32,
    #min_child_weight = 0.05,
    max_delta_step = 0,
    subsample= 0.7,
    colsample_bytree= 0.7,
    refresh_leaf = 1,
    eval_metric= 'auc',
    tree_method = 'hist')

In [1257]:
model.fit(X=X_train, y=y_train, verbose=1)

XGBClassifier(base_score=0.7, binary='binary:logistic', booster='gbtree',
       colsample_bylevel=1, colsample_bytree=0.7, eta=0.1,
       eval_metric='auc', gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, max_leaves=32, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=4, num_round=107,
       objective='binary:logistic', random_state=0, refresh_leaf=1,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.7, tree_method='hist')

In [1230]:
val_pred = model.predict_proba(X_val)

In [1231]:
roc_auc_score(y_val,val_pred[:,1])#0.748857727149153 0.7467943004410285 0.7470185745213075 0.7470180486735226 0.7470169969779527 0.7452065030543868 0.7467943004410285 0.7466775622327708 0.7467943004410285

0.748857727149153

In [1044]:
model.fit(X=X, y=y, verbose=1)

XGBClassifier(base_score=0.7, binary='binary:logistic', booster='gbtree',
       colsample_bylevel=1, colsample_bytree=0.7, eta=0.1,
       eval_metric='auc', gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, max_leaves=32, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=4, num_round=107,
       objective='binary:logistic', random_state=0, refresh_leaf=1,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.7, tree_method='hist')

In [1045]:
X_test = test.drop(['0'], axis=1)
y_test = test['0']

In [1046]:
predict_xgb = model.predict_proba(X_test)

In [1047]:
test_index = pd.read_csv('test.csv', sep='\t')
test_index = test_index['Unnamed: 0']

In [1048]:
submition = pd.DataFrame({'_ID_': test_index, '_VAL_': np.zeros_like(test_index)})
submition['_VAL_'] = predict_xgb[:,1]
submition.to_csv('submission_xgb.csv', index=False)

# cv 0.748857727149153
# LB 0.75328152

# LightGBM

In [1259]:
parameters = {
    'objective': 'binary',
    'boost':'gbdt',
    'learning_rate': 0.1,
    'learning_rate': 0.1,
    'num_threads': 4,
    'metric': 'auc',
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 15,
    'cat_smooth': 10,
    'min_data_per_group': 40,
    'max_depth':4,
    'num_leaves' : 25,
    'n_trees':140,
}
n_rounds = 10000

In [1260]:
lgb_train = lgb.Dataset(X, label=y, free_raw_data=False)

In [1199]:
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf.split(X, y), early_stopping_rounds=10, verbose_eval=1, )

[1]	cv_agg's auc: 0.665225 + 0.00211299
[2]	cv_agg's auc: 0.68213 + 0.00602095
[3]	cv_agg's auc: 0.690216 + 0.00601505
[4]	cv_agg's auc: 0.69333 + 0.00459103
[5]	cv_agg's auc: 0.696374 + 0.00588247
[6]	cv_agg's auc: 0.69787 + 0.00578632
[7]	cv_agg's auc: 0.700498 + 0.00541619
[8]	cv_agg's auc: 0.701822 + 0.00496066
[9]	cv_agg's auc: 0.703757 + 0.00497327
[10]	cv_agg's auc: 0.70573 + 0.00527157
[11]	cv_agg's auc: 0.707065 + 0.00602357
[12]	cv_agg's auc: 0.708264 + 0.00578034
[13]	cv_agg's auc: 0.709593 + 0.00513989
[14]	cv_agg's auc: 0.710494 + 0.00606343
[15]	cv_agg's auc: 0.710683 + 0.00624467
[16]	cv_agg's auc: 0.712006 + 0.00648105
[17]	cv_agg's auc: 0.712995 + 0.00649271
[18]	cv_agg's auc: 0.713757 + 0.00648126
[19]	cv_agg's auc: 0.714066 + 0.00661968
[20]	cv_agg's auc: 0.71469 + 0.00666435
[21]	cv_agg's auc: 0.715831 + 0.00611832
[22]	cv_agg's auc: 0.716946 + 0.00589303
[23]	cv_agg's auc: 0.718012 + 0.00567794
[24]	cv_agg's auc: 0.719256 + 0.00599265
[25]	cv_agg's auc: 0.719763 + 

In [1200]:
max(result['auc-mean']), len(result['auc-mean'])

(0.7417662406887991, 140)

In [1201]:
0.7417662406887991

0.7417662406887991

In [1261]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.12, random_state=42)

In [1262]:
train_data = lgb.Dataset(X_train, y_train)

In [1263]:
#X_train,y_train

In [1264]:
n_rounds = 140

In [1265]:
bst = lgb.train(parameters, train_data, n_rounds)

In [1244]:
val_pred = bst.predict(X_val,num_iteration=bst.best_iteration)

In [1245]:
val_pred

array([0.18435973, 0.12721367, 0.10777017, ..., 0.06167979, 0.12118198,
       0.5146969 ])

In [1246]:
roc_auc_score(y_val,val_pred)# 0.7470185745213075 0.7470180486735226 0.7470169969779527 0.7452065030543868 0.7467943004410285 0.7466775622327708 0.7467943004410285

0.7445762744841302

In [1122]:
7467785250074802

7467785250074802

In [1163]:
X_test = test.drop(['0'], axis=1)
y_test = test['0']

In [1164]:
predict_lgb = bst.predict(X_test,num_iteration=bst.best_iteration)

In [1165]:
test_index = pd.read_csv('test.csv', sep='\t')
test_index = test_index['Unnamed: 0']

In [1166]:
submition = pd.DataFrame({'_ID_': test_index, '_VAL_': np.zeros_like(test_index)})
submition['_VAL_'] = predict_lgb
submition.to_csv('submission_lgb_2.csv', index=False)

# cv 7445762744841302
# lb 0.74997541

# CatBoost

In [1211]:
import catboost as ctb

In [1212]:
ctb_data = ctb.Pool(X,y)

In [1213]:
parameters = {
   'eval_metric': 'AUC',
}

In [1214]:
ctb.cv(ctb_data, parameters, stratified=True, seed=42, logging_level='Verbose')

0:	learn: 0.6326269	test: 0.6279181	best: 0.6279181 (0)	total: 587ms	remaining: 9m 46s
1:	learn: 0.6699014	test: 0.6662706	best: 0.6662706 (1)	total: 952ms	remaining: 7m 54s
2:	learn: 0.6778677	test: 0.6721965	best: 0.6721965 (2)	total: 1.32s	remaining: 7m 18s
3:	learn: 0.6797696	test: 0.6742282	best: 0.6742282 (3)	total: 1.68s	remaining: 6m 59s
4:	learn: 0.6851759	test: 0.6777598	best: 0.6777598 (4)	total: 2.04s	remaining: 6m 46s
5:	learn: 0.6867105	test: 0.6787802	best: 0.6787802 (5)	total: 2.44s	remaining: 6m 44s
6:	learn: 0.6887171	test: 0.6799080	best: 0.6799080 (6)	total: 2.83s	remaining: 6m 41s
7:	learn: 0.6898727	test: 0.6804936	best: 0.6804936 (7)	total: 3.2s	remaining: 6m 36s
8:	learn: 0.6902141	test: 0.6808548	best: 0.6808548 (8)	total: 3.57s	remaining: 6m 32s
9:	learn: 0.6909350	test: 0.6808837	best: 0.6808837 (9)	total: 3.93s	remaining: 6m 29s
10:	learn: 0.6908245	test: 0.6804729	best: 0.6808837 (9)	total: 4.32s	remaining: 6m 28s
11:	learn: 0.6902957	test: 0.6803632	best: 

93:	learn: 0.7287047	test: 0.7122444	best: 0.7122444 (93)	total: 35.4s	remaining: 5m 41s
94:	learn: 0.7291051	test: 0.7124937	best: 0.7124937 (94)	total: 35.8s	remaining: 5m 40s
95:	learn: 0.7293953	test: 0.7127500	best: 0.7127500 (95)	total: 36.2s	remaining: 5m 40s
96:	learn: 0.7296015	test: 0.7127894	best: 0.7127894 (96)	total: 36.6s	remaining: 5m 40s
97:	learn: 0.7298367	test: 0.7129687	best: 0.7129687 (97)	total: 36.9s	remaining: 5m 39s
98:	learn: 0.7300954	test: 0.7131587	best: 0.7131587 (98)	total: 37.3s	remaining: 5m 39s
99:	learn: 0.7303666	test: 0.7134380	best: 0.7134380 (99)	total: 37.7s	remaining: 5m 39s
100:	learn: 0.7306050	test: 0.7136013	best: 0.7136013 (100)	total: 38.1s	remaining: 5m 38s
101:	learn: 0.7308729	test: 0.7136989	best: 0.7136989 (101)	total: 38.4s	remaining: 5m 38s
102:	learn: 0.7311173	test: 0.7139252	best: 0.7139252 (102)	total: 38.8s	remaining: 5m 38s
103:	learn: 0.7313909	test: 0.7140051	best: 0.7140051 (103)	total: 39.2s	remaining: 5m 37s
104:	learn: 0

184:	learn: 0.7507890	test: 0.7238753	best: 0.7238753 (184)	total: 1m 10s	remaining: 5m 9s
185:	learn: 0.7510123	test: 0.7239782	best: 0.7239782 (185)	total: 1m 10s	remaining: 5m 8s
186:	learn: 0.7512438	test: 0.7239567	best: 0.7239782 (185)	total: 1m 10s	remaining: 5m 8s
187:	learn: 0.7515118	test: 0.7240301	best: 0.7240301 (187)	total: 1m 11s	remaining: 5m 8s
188:	learn: 0.7516262	test: 0.7240542	best: 0.7240542 (188)	total: 1m 11s	remaining: 5m 7s
189:	learn: 0.7518215	test: 0.7241163	best: 0.7241163 (189)	total: 1m 12s	remaining: 5m 7s
190:	learn: 0.7520046	test: 0.7241707	best: 0.7241707 (190)	total: 1m 12s	remaining: 5m 7s
191:	learn: 0.7522058	test: 0.7242094	best: 0.7242094 (191)	total: 1m 12s	remaining: 5m 7s
192:	learn: 0.7524126	test: 0.7242515	best: 0.7242515 (192)	total: 1m 13s	remaining: 5m 6s
193:	learn: 0.7525168	test: 0.7242985	best: 0.7242985 (193)	total: 1m 13s	remaining: 5m 6s
194:	learn: 0.7526709	test: 0.7243774	best: 0.7243774 (194)	total: 1m 14s	remaining: 5m 5s

274:	learn: 0.7661691	test: 0.7291263	best: 0.7291263 (274)	total: 1m 45s	remaining: 4m 37s
275:	learn: 0.7663419	test: 0.7291587	best: 0.7291587 (275)	total: 1m 45s	remaining: 4m 36s
276:	learn: 0.7664584	test: 0.7291992	best: 0.7291992 (276)	total: 1m 45s	remaining: 4m 36s
277:	learn: 0.7666134	test: 0.7292503	best: 0.7292503 (277)	total: 1m 46s	remaining: 4m 36s
278:	learn: 0.7667211	test: 0.7292695	best: 0.7292695 (278)	total: 1m 46s	remaining: 4m 36s
279:	learn: 0.7668769	test: 0.7293081	best: 0.7293081 (279)	total: 1m 47s	remaining: 4m 35s
280:	learn: 0.7670522	test: 0.7293569	best: 0.7293569 (280)	total: 1m 47s	remaining: 4m 35s
281:	learn: 0.7671758	test: 0.7293968	best: 0.7293968 (281)	total: 1m 48s	remaining: 4m 35s
282:	learn: 0.7674313	test: 0.7294172	best: 0.7294172 (282)	total: 1m 48s	remaining: 4m 34s
283:	learn: 0.7676636	test: 0.7294669	best: 0.7294669 (283)	total: 1m 48s	remaining: 4m 34s
284:	learn: 0.7678643	test: 0.7295351	best: 0.7295351 (284)	total: 1m 49s	remain

364:	learn: 0.7812495	test: 0.7322038	best: 0.7322038 (364)	total: 2m 24s	remaining: 4m 11s
365:	learn: 0.7813421	test: 0.7322318	best: 0.7322318 (365)	total: 2m 25s	remaining: 4m 11s
366:	learn: 0.7814979	test: 0.7322376	best: 0.7322376 (366)	total: 2m 25s	remaining: 4m 10s
367:	learn: 0.7817030	test: 0.7321914	best: 0.7322376 (366)	total: 2m 25s	remaining: 4m 10s
368:	learn: 0.7818662	test: 0.7321832	best: 0.7322376 (366)	total: 2m 26s	remaining: 4m 10s
369:	learn: 0.7820506	test: 0.7322148	best: 0.7322376 (366)	total: 2m 26s	remaining: 4m 9s
370:	learn: 0.7822033	test: 0.7322833	best: 0.7322833 (370)	total: 2m 27s	remaining: 4m 9s
371:	learn: 0.7823702	test: 0.7323349	best: 0.7323349 (371)	total: 2m 27s	remaining: 4m 8s
372:	learn: 0.7825519	test: 0.7323373	best: 0.7323373 (372)	total: 2m 27s	remaining: 4m 8s
373:	learn: 0.7827432	test: 0.7324171	best: 0.7324171 (373)	total: 2m 28s	remaining: 4m 8s
374:	learn: 0.7829107	test: 0.7324706	best: 0.7324706 (374)	total: 2m 28s	remaining: 

454:	learn: 0.7956712	test: 0.7335817	best: 0.7335817 (454)	total: 3m 2s	remaining: 3m 38s
455:	learn: 0.7958614	test: 0.7336190	best: 0.7336190 (455)	total: 3m 3s	remaining: 3m 38s
456:	learn: 0.7960126	test: 0.7336559	best: 0.7336559 (456)	total: 3m 3s	remaining: 3m 38s
457:	learn: 0.7961622	test: 0.7336946	best: 0.7336946 (457)	total: 3m 4s	remaining: 3m 37s
458:	learn: 0.7963601	test: 0.7337061	best: 0.7337061 (458)	total: 3m 4s	remaining: 3m 37s
459:	learn: 0.7964946	test: 0.7337365	best: 0.7337365 (459)	total: 3m 5s	remaining: 3m 37s
460:	learn: 0.7966127	test: 0.7337003	best: 0.7337365 (459)	total: 3m 5s	remaining: 3m 37s
461:	learn: 0.7967787	test: 0.7336475	best: 0.7337365 (459)	total: 3m 6s	remaining: 3m 36s
462:	learn: 0.7969259	test: 0.7336572	best: 0.7337365 (459)	total: 3m 6s	remaining: 3m 36s
463:	learn: 0.7970409	test: 0.7337197	best: 0.7337365 (459)	total: 3m 7s	remaining: 3m 36s
464:	learn: 0.7971690	test: 0.7337838	best: 0.7337838 (464)	total: 3m 7s	remaining: 3m 35s

544:	learn: 0.8083110	test: 0.7348669	best: 0.7348959 (538)	total: 3m 44s	remaining: 3m 7s
545:	learn: 0.8084443	test: 0.7348949	best: 0.7348959 (538)	total: 3m 44s	remaining: 3m 6s
546:	learn: 0.8085906	test: 0.7348921	best: 0.7348959 (538)	total: 3m 45s	remaining: 3m 6s
547:	learn: 0.8087253	test: 0.7349244	best: 0.7349244 (547)	total: 3m 45s	remaining: 3m 6s
548:	learn: 0.8088130	test: 0.7349010	best: 0.7349244 (547)	total: 3m 46s	remaining: 3m 5s
549:	learn: 0.8089573	test: 0.7349018	best: 0.7349244 (547)	total: 3m 46s	remaining: 3m 5s
550:	learn: 0.8090896	test: 0.7349245	best: 0.7349245 (550)	total: 3m 46s	remaining: 3m 4s
551:	learn: 0.8091979	test: 0.7349416	best: 0.7349416 (551)	total: 3m 47s	remaining: 3m 4s
552:	learn: 0.8093642	test: 0.7349430	best: 0.7349430 (552)	total: 3m 47s	remaining: 3m 4s
553:	learn: 0.8095309	test: 0.7349098	best: 0.7349430 (552)	total: 3m 48s	remaining: 3m 3s
554:	learn: 0.8096596	test: 0.7349152	best: 0.7349430 (552)	total: 3m 48s	remaining: 3m 3s

634:	learn: 0.8195473	test: 0.7361962	best: 0.7362165 (633)	total: 4m 24s	remaining: 2m 31s
635:	learn: 0.8196549	test: 0.7361858	best: 0.7362165 (633)	total: 4m 24s	remaining: 2m 31s
636:	learn: 0.8197992	test: 0.7361796	best: 0.7362165 (633)	total: 4m 24s	remaining: 2m 30s
637:	learn: 0.8199159	test: 0.7361423	best: 0.7362165 (633)	total: 4m 25s	remaining: 2m 30s
638:	learn: 0.8200381	test: 0.7361465	best: 0.7362165 (633)	total: 4m 25s	remaining: 2m 30s
639:	learn: 0.8201885	test: 0.7361259	best: 0.7362165 (633)	total: 4m 26s	remaining: 2m 29s
640:	learn: 0.8203047	test: 0.7361501	best: 0.7362165 (633)	total: 4m 26s	remaining: 2m 29s
641:	learn: 0.8204520	test: 0.7361464	best: 0.7362165 (633)	total: 4m 27s	remaining: 2m 29s
642:	learn: 0.8205729	test: 0.7361803	best: 0.7362165 (633)	total: 4m 27s	remaining: 2m 28s
643:	learn: 0.8206868	test: 0.7361889	best: 0.7362165 (633)	total: 4m 28s	remaining: 2m 28s
644:	learn: 0.8208312	test: 0.7361837	best: 0.7362165 (633)	total: 4m 28s	remain

KeyboardInterrupt: 

In [1215]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': 'AUC',
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': True
}

In [ ]:
#clf = CatBoostClassifier(learning_rate=0.1, eval_metric= 'AUC',depth = 10,iterations=500, random_seed=0, logging_level='Silent')

In [ ]:
clf = CatBoostClassifier(**params)

In [1216]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.12, random_state=42)

In [1217]:
train_pool = Pool(X_train, y_train)
validate_pool = Pool(X_val, y_val)

In [ ]:
clf.fit(train_pool, eval_set=validate_pool)

In [1218]:
best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})

In [1219]:
best_model = CatBoostClassifier(**best_model_params)

In [1220]:
best_model.get_params()

{'eval_metric': 'AUC',
 'iterations': 500,
 'learning_rate': 0.1,
 'logging_level': 'Silent',
 'loss_function': 'Logloss',
 'random_seed': 42,
 'use_best_model': True}

In [1221]:
best_model.fit(train_pool, eval_set=validate_pool);

In [1222]:
print('Simple model validation accuracy: {:.4}'.format(
    roc_auc_score(y_val, clf.predict_proba(X_val)[:,1])
))

Simple model validation accuracy: 0.734


In [1223]:
print('')

print('Best model validation accuracy: {:.4}'.format(
    roc_auc_score(y_val, best_model.predict_proba(X_val)[:,1])
))


Best model validation accuracy: 0.7448


In [1224]:
predict_cat = best_model.predict_proba(X_test)

In [1225]:
test_index = pd.read_csv('test.csv', sep='\t')
test_index = test_index['Unnamed: 0']

In [1226]:
submition = pd.DataFrame({'_ID_': test_index, '_VAL_': np.zeros_like(test_index)})
submition['_VAL_'] = predict_cat[:,1]
submition.to_csv('submission_cat.csv', index=False)

# cv 0.7448
lb 0.74842713

# Ensamble

In [1266]:
lr_pred_cv = []
rf_pred_cv = []
bg_pred_cv = []
k=0
for train, val in skf.split(X, y):
    k+=1
    print(f"Iteration: {k}")
    model.fit(X.iloc[train], y[train])
    print("lgb_s")
    bst.train(model.Dataset(X.iloc[train], y[train]))
    best_model.fit(Pool(X.iloc[train], y[train]))
    rf_pred_cv.append([y[val], model.predict_proba(X.iloc[val])[:,1]])
    lr_pred_cv.append([y[val], lgb.predict_proba(X.iloc[val])[:,1]])
    best_model.append([y[val], best_bgc.predict_proba(X.iloc[val])[:,1]])

Iteration: 1
lgb_s


AttributeError: 'Booster' object has no attribute 'train'